In [38]:
import sys; sys.path.insert(0, '..')
from database.dataset import *
import pyforest
import matplotlib.pyplot as plt

In [2]:
def easy_plot(i):
    plt.figure()
    plt.imshow(db[i]['img'], cmap='gray' )
    plt.imshow(db[i]['lesion_mask'])
    plt.axis('off')
    plt.show()

In [3]:
db = INBreast_Dataset(
        return_lesions_mask=True,
        level='image',
        max_lesion_diam_mm=1.0,
        normalize=None,
        n_jobs=-1,
)

In [4]:
db.df.loc[0]

img_id                                                      20586908
n_rois                                                            42
side                                                               R
view                                                              CC
filename           /home/acortinau/projects/calc-det/data/INbreas...
acr                                                                2
artifact                                                       False
birads                                                             2
case_id                                             6c613a14b80a8591
finding_notes                           nodulo mama direita + micros
lesion_annot                                                     yes
pectoral_muscle                                                False
mass                                                            True
micros                                                          True
distortion                        

In [9]:
db =  INBreast_Dataset(
        return_lesions_mask=True,
        level='rois',
        max_lesion_size_mm=1.0,
        extract_patches=True,
        extract_patches_method='all',  # 'centered'
        patch_size=512,
        stride=256,
        min_breast_fraction_roi=0.5,
        normalize=None,
        n_jobs=-1)

100%|██████████| 310/310 [01:12<00:00,  4.27it/s]


In [34]:
def get_patches_from_candidates(image_array: np.ndarray, lesion_candidates: list[tuple], patch_size: int):
    """Returns patches of given size centered at each lesion center coordinates.

    Args:
        image_array (np.ndarray): image or mask to extract patches.
        lesion_candidates (list): corresponding list of lesion candidates. Each candidate be a tuple of int coordinates (row,column).
        patch_size (int): dimension of the extracted patch (patch_size,patch_size).

    Returns:
        candidate_patches (np.ndarray): extracted patches array. Dim (no_patches,patch_size,patch_size)
    """
    no_lesions = np.shape(lesion_candidates)[0]  
    candidates_patches = np.zeros((no_lesions,patch_size,patch_size))
       
    for k,(x_coord,y_coord) in enumerate(lesion_candidates):
        
        patch_half_size = int(patch_size / 2)
        
        # Get the coordinates of the patch centered in the lesion
        patch_x1 = x_coord - patch_half_size
        patch_y1 = y_coord - patch_half_size
        if patch_x1 < 0:
            patch_x1 = 0
            patch_x2 = patch_size
        else:
            patch_x2 = x_coord + patch_size - patch_half_size
        if patch_y1 < 0:
            patch_y1 = 0
            patch_y2 = patch_size
        else:
            patch_y2 = y_coord + patch_size - patch_half_size

        if patch_x2 > image_array.shape[1]:
            image_array = np.pad(
                image_array, ((0, 0), (0, patch_size)), mode='constant', constant_values=0
            )
            
        if patch_y2 > image_array.shape[0]:
            image_array = np.pad(
                image_array, ((0, patch_size), (0, 0)), mode='constant', constant_values=0
            )
            
        # Crop the patch
        candidates_patches[k] = image_array[patch_y1:patch_y2, patch_x1:patch_x2]
        
    return candidates_patches


In [35]:
blob_coord = list(db.rois_df.loc[db.rois_df.img_id == db.df.loc[0,'img_id'],'center_crop'])



In [51]:
can_patches = get_patches_from_candidates(db[0]['img'],blob_coord,200)

In [42]:
db = INBreast_Dataset(
        return_lesions_mask=True,
        level='rois',
        max_lesion_size_mm=1.0,
        extract_rois=False,
        extract_rois_method='all',  # 'centered'
        roi_size=256,
        stride=256,
        min_breast_fraction_roi=0.8,
        normalize=None,
        n_jobs=-1,
)